# Imports

In [51]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import cohen_kappa_score
import torch
from tqdm import tqdm
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, logging
import glob
import os


# 🔍 Prometheus LLM-as-a-Judge

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

prometheur_model_name = "prometheus-eval/prometheus-7b-v2.0"

prometheus_tokenizer = AutoTokenizer.from_pretrained(
    prometheur_model_name,
    trust_remote_code=True
)

prometheus_model = AutoModelForCausalLM.from_pretrained(
    prometheur_model_name,
    trust_remote_code=True, 
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload_prometheus"
)

judge = pipeline("text-generation", model=prometheus_model, tokenizer=prometheus_tokenizer)


Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.15s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import re
from tqdm import tqdm
import gc
import csv

# === 1. Carica i dati ===
df_base = pd.read_csv("inputs/dataset_human_eval.csv").head(15)  # usa .head(N) per limitare
df_mistral = pd.read_csv("outputs/dataset_with_mistral_translations.csv").head(15)
df_nnlb = pd.read_csv("outputs/dataset_with_translation_NNLB.csv").head(15)

df_base["mistral"] = df_mistral["generated_translation"]
df_base["mistralHS"] = df_mistral["score_human"]
df_base["nnlb"] = df_nnlb["generated_translation"]
df_base["nnlbHS"] = df_nnlb["score_human"]
# === 2. Carica il modello Prometheus ===
model_name = "prometheus-eval/prometheus-7b-v2.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload_prometheus",
    offload_buffers=True
)
model.eval()

# === 3. Rubrica e prompt ===

def build_judge_prompt(original, human_translation, model_translation):
    return f"""
You are a translation evaluator.

Original:
{original}

Human:
{human_translation}

Model:
{model_translation}
Your task is to assign a score from 1 to 5 that reflects how well the model's translation preserves the original meaning:
score 1 = Completely wrong
score 2 = Major errors
score 3 = Understandable but flawed
score 4 = Mostly correct
score 5 = Perfectly accurate

Always answer with just the number:""".strip()




def estrai_punteggio(output):
    match = re.search(r"number:\s*(\d)", output)
    return int(match.group(1)) if match else None

# === 4. Valutazione ===
score_as = []
score_bs = []
winners = []

for _, row in tqdm(df_base.iterrows(), total=len(df_base)):
    for label, translation, score_list in [("A", row["mistral"], score_as), ("B", row["nnlb"], score_bs)]:
        prompt = build_judge_prompt(row["Sentence"], row["HumanEval"], translation)
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False)
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        print(output_text)
        score = estrai_punteggio(output_text)
        score_list.append(score)

        # Pulisce memoria
        del inputs, output_ids
        torch.cuda.empty_cache()
        gc.collect()

    score_a = score_as[-1]
    score_b = score_bs[-1]
    if score_a is None or score_b is None:
        winners.append("?")
    elif score_a > score_b:
        winners.append("A")
    elif score_b > score_a:
        winners.append("B")
    else:
        winners.append("=")

# === 5. Salvataggio ===
df_base["score_a"] = score_as
df_base["score_b"] = score_bs
df_base["winner"] = winners

df_base.to_csv("outputs/prometheus_eval.csv", index=False, quoting=csv.QUOTE_MINIMAL)
print("✅ File salvato in outputs/prometheus_eval.csv")


Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.46s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
  0%|          | 0/15 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a translation evaluator.

Original:
quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno

Human:
Quella guerra fu condotta bene perchè etc. Dall’altra parte, Aiace era un cavaliere franco e valoroso nelle armi, di grande statura, ma non possedeva grande saggezza.

Model:
Quella guerra fu combattuta molto bene perché, eccetto che, etc. E dall'altra parte Aiace era un cavaliere francese e valente in armi, di grande statura, ma non aveva grande saggezza.
Your task is to assign a score from 1 to 5 that reflects how well the model's translation preserves the original meaning:
score 1 = Completely wrong
score 2 = Major errors
score 3 = Understandable but flawed
score 4 = Mostly correct
score 5 = Perfectly accurate

Always answer with just the number: 5


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a translation evaluator.

Original:
quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno

Human:
Quella guerra fu condotta bene perchè etc. Dall’altra parte, Aiace era un cavaliere franco e valoroso nelle armi, di grande statura, ma non possedeva grande saggezza.

Model:
E d'altra parte, Aiaces era un cavaliere franco e dotato di armi di grande talento, ma non era pieno di grande saggezza.
Your task is to assign a score from 1 to 5 that reflects how well the model's translation preserves the original meaning:
score 1 = Completely wrong
score 2 = Major errors
score 3 = Understandable but flawed
score 4 = Mostly correct
score 5 = Perfectly accurate

Always answer with just the number: 4


  7%|▋         | 1/15 [00:03<00:47,  3.40s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

You are a translation evaluator.

Original:
Io mi ricordo (ch. 347) che essendo adirato scapigliai la mia donna. Ohi, quanti dì questa ira mi tolse!

Human:
Ricordo che, essendo arrabbiato, scompigliai i capelli della mia donna. Ahimè, quanti giorni persi a causa di quell’ira!

Model:
Ricordo che, arrabbiato, la mia donna irritai. Oh, quanti giorni mi tolse quell’ira!
Your task is to assign a score from 1 to 5 that reflects how well the model's translation preserves the original meaning:
score 1 = Completely wrong
score 2 = Major errors
score 3 = Understandable but flawed
score 4 = Mostly correct
score 5 = Perfectly accurate

Always answer with just the number: 5


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a translation evaluator.

Original:
Io mi ricordo (ch. 347) che essendo adirato scapigliai la mia donna. Ohi, quanti dì questa ira mi tolse!

Human:
Ricordo che, essendo arrabbiato, scompigliai i capelli della mia donna. Ahimè, quanti giorni persi a causa di quell’ira!

Model:
Mi ricordo (cfr. 347) che quando fui infuriato, fui in fuga con mia moglie. Oh, quanti giorni mi ha spinto questa ira!
Your task is to assign a score from 1 to 5 that reflects how well the model's translation preserves the original meaning:
score 1 = Completely wrong
score 2 = Major errors
score 3 = Understandable but flawed
score 4 = Mostly correct
score 5 = Perfectly accurate

Always answer with just the number: 5


100%|██████████| 15/15 [00:43<00:00,  2.92s/it]

✅ File salvato in outputs/prometheus_scores.csv


In [23]:
print(output_text)

You are a translation evaluator.

Original:
Io mi ricordo (ch. 347) che essendo adirato scapigliai la mia donna. Ohi, quanti dì questa ira mi tolse!

Human:
Ricordo che, essendo arrabbiato, scompigliai i capelli della mia donna. Ahimè, quanti giorni persi a causa di quell’ira!

Model:
Mi ricordo (cfr. 347) che quando fui infuriato, fui in fuga con mia moglie. Oh, quanti giorni mi ha spinto questa ira!

Evaluate how well the model's translation preserves the original meaning.

Semantic fidelity of the translation
Score 1: Meaning completely lost or unrelated to the original.
Score 2: Major distortions or omissions; the translation misleads the reader.
Score 3: Main idea is conveyed, but with several inaccuracies or confusing parts.
Score 4: Mostly correct. Minor issues, but the meaning is clear.
Score 5: Fully accurate. Captures all meaning in a fluent, faithful way.

Reply with:
Score (1–5): 5

The


# 📊 Calcolo concordanza

In [ ]:
df_base = pd.read_csv("outputs/prometheus_eval.csv")

In [50]:
from sklearn.metrics import cohen_kappa_score, classification_report, confusion_matrix

def evaluate_concordance(y_true, y_pred):
    print("== Valori unici ==")
    print("Umano:", sorted(set(y_true)))
    print("Modello:", sorted(set(y_pred)))
    
    if len(set(y_true)) < 2 or len(set(y_pred)) < 2:
        print("⚠️ Non abbastanza variabilità per calcolare Cohen's Kappa.")
        return
    
    print("\n== Cohen’s Kappa ==")
    print(f"{cohen_kappa_score(y_true, y_pred):.3f}")


print("MISTRAL")
evaluate_concordance(df_base["mistralHS"], df_base["score_a"])

print("NNLB")
evaluate_concordance(df_base["nnlbHS"], df_base["score_b"])


MISTRAL
== Valori unici ==
Umano: [2, 3, 4, 5]
Modello: [1, 3, 5]

== Cohen’s Kappa ==
0.137
NNLB
== Valori unici ==
Umano: [2, 3, 4, 5]
Modello: [1, 2, 4, 5]

== Cohen’s Kappa ==
0.052
